# Context
* When an applicant submits a loan it may be either Accepted or Rejected.

, there are 3 possible loan statuses that may occur:
    1. Currently active - loan is still being paid off and it is before the due date
    2. Fully Paid - The loan was fully paid in the allotted time
    3. Default - The applicant was unable to pay the loan fully before the due date and the loan is considered "defaulted"

# Data Collection

In this notebook, we will create the dataset by pulling from LendersClub data and then take the necessary measures to **Clean the Data** and apply imputation techniques 

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import kagglehub # For the data
import os 

### Load the dataset

In [14]:
# Download latest version of kagglehub
path = kagglehub.dataset_download("wordsforthewise/lending-club")
os.chdir(r"C:\Users\henry\.cache\kagglehub\datasets\wordsforthewise\lending-club\versions\3") # Change into path containing the data


accepted_df = pd.read_csv("accepted_2007_to_2018Q4.csv.gz")
rejected_df = pd.read_csv("rejected_2007_to_2018Q4.csv.gz")

C:\Users\henry\AppData\Local\Temp\ipykernel_9588\707967294.py:6: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df = pd.read_csv("accepted_2007_to_2018Q4.csv.gz")


### EDA